<a href="https://colab.research.google.com/github/noorhaq/Google_Colab/blob/master/Sad_Happy_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

#Directory with happy training data
train_happy_dir = os.path.join('/tmp/h-or-s/happy');

#Directory with sad training data
train_sad_dir = os.path.join('/tmp/h-or-s/sad')

##To Know Total Number of Training Images. In this case 40 happy, 40 sad.
print('Total Training Happy Images :', len(os.listdir(train_happy_dir)))
print('Total Training Sad Images :', len(os.listdir(train_sad_dir)))

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs ={}):
    if(logs.get('accuracy')> DESIRED_ACCURACY):
      print("\nReached 99.9% accuracy, Cancelling Training.\n")
      self.model.stop_training =True

callbacks = myCallback()

In [36]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
 tf.keras.layers.Conv2D(16,(3,3), activation= 'relu', input_shape=(150,150,3)),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(32,(3,3), activation= 'relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(32,(3,3), activation= 'relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation = 'relu'),
tf.keras.layers.Dense(1, activation ='sigmoid')
])

model.summary()
from tensorflow.keras.optimizers import RMSprop

model.compile(loss = 'binary_crossentropy',
              optimizer =RMSprop(lr =0.001),
              metrics = ['accuracy'])


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 9248)             

In [37]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)


train_generator = train_datagen.flow_from_directory(
       "/tmp/h-or-s/",
       target_size = (150,150),
       batch_size=10,
       class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [38]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      callbacks=[callbacks]
)    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
8/8 [==============================] - 0s 20ms/step - loss: 1.1466 - accuracy: 0.5500
Epoch 2/15
8/8 [==============================] - 0s 20ms/step - loss: 0.6211 - accuracy: 0.6750
Epoch 3/15
8/8 [==============================] - 0s 21ms/step - loss: 0.4590 - accuracy: 0.8125
Epoch 4/15
8/8 [==============================] - 0s 21ms/step - loss: 0.4703 - accuracy: 0.8125
Epoch 5/15
8/8 [==============================] - 0s 21ms/step - loss: 0.1079 - accuracy: 0.9625
Epoch 6/15
8/8 [==============================] - 0s 20ms/step - loss: 0.1197 - accuracy: 0.9250
Epoch 7/15
8/8 [==============================] - 0s 21ms/step - loss: 0.0617 - accuracy: 0.9750
Epoch 8/15
8/8 [==============================] - 0s 21ms/step - loss: 0.0373 - accuracy: 0.9875
Epoch 9/15
7/8 [=========================>....] - ETA: 0s - loss: 0.0183 - accuracy: 1.0000
Reached 99.9% accuracy, Cancelling Training.

8/8 [==============================] - 0s 20ms/step - loss: 0.0190 - accuracy: 1.0000
